In [3]:
import pandas as pd
from typing import List

### Reading the Data and Initial Stats

In [4]:
movies_df: pd.DataFrame = pd.read_csv('input/all_movies.csv')
movies_df.sample(20)

,movieId,title,genres
54917,186219,The Phantom of the Opera (1962),Drama|horror|MYSTERY
27573,122942,The Identical (2014),Drama
57252,191741,Golden Slumber (2010),Action|comedy|DRAMA|mystery|Thriller
22237,106032,Chastity Bites (2013),Comedy|horror
23660,110750,Heaven Is for Real (2014),Drama
25966,118330,Blood (2012),Crime|drama|THRILLER
13429,65909,Maria (2003),Drama
27845,123559,Spitfire (1995),(no genres listed)
35615,142663,Babes in Toyland (1997),Animation|children
56822,190539,Odnoklassniki.ru: The Magic Laptop (2013),Comedy|romance


In [5]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|animation|CHILDREN|comedy|Fantasy
1,2,Jumanji (1995),Adventure|children|FANTASY
2,3,Grumpier Old Men (1995),Comedy|romance
3,4,Waiting to Exhale (1995),Comedy|drama|ROMANCE
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed)
58094,193878,Les tribulations d'une caissière (2011),Comedy
58095,193880,Her Name Was Mumu (2016),Drama
58096,193882,Flora (2017),Adventure|drama|HORROR|sci-fi


## Step 1: Cleaning
You shouldn't necessarily assume that your data is good.  It could be very sparse and not have much there. There could be duplication, poorly recorded or empty values, or with large text there could be a lot of garbage in there if it was an open text field

Some things we'll do here:
1. Get the number of rows and columns by looking at the shape
2. Determine the number of non-null rows for each given column (This would tell us if a column is especially sparse)
3. Check for duplicate rows

As we find things that need to be cleaned (bad text, duplicates etc.) we will write tested cleaning functions to cleanup our input data.

#### Let's do some cleanup.

### Checking for Duplicates

#### We need to define what a "duplicate" is.

Get a series with True or False that indicates if the title was duplicated or not.

Locate all records where the title was duplicated.

Get a series with True of False that indicates if the title and genre were both duplciated.

Locate all records where the title and genre were duplicated

We'll need a list of titles from each category

We have two lists of movies:
1. Has all the titles where the title was duplicated - in this list some titles represent records that have duplicated genre strings and some represent records that have different genre string.
2. Has all the titles where the title and genre was duplicated - in this list the title represents records that have duplicated genre strings.

With these two lists how can we get a list that is ONLY the titles where the title was duplicated but the genre was not?

Now we can locate an example using the titles in our list.

## Step 2: Feature Preparation

We can focus on the genres to recommend movies.  Let's prepare our genres list. First we need to group by movie.

Clean up the genres lists.

Let's think about our recommendation engine now again.  Let's say that we want to recommend movies to by recommending the movies with the most similar genres list.

In order to to use TF IDF we need a list of all the "words" (genres) used in our corpus.  This is easy for us to do.  We can make a list of all the genres by:
1. Creating a column with a list of genres
2. Grouping by the genre
3. Aggregating the results
4. Transforming the resulting series into a list of genres

Group by Genre

create a list of genres

Let's turn our genres column into a space separated list of genres (as if they were words in a document)

reset our index again

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(vocabulary=all_genres)
tfidf_matrix = tf.fit_transform(movies_ready_for_vectorizer['genres'])

To see what our matrix looks like we can do the following:

In [41]:
pd.DataFrame(tfidf_matrix.toarray())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,0.0,0.0,0.000000,0.000000,0.775664,0.000000,0.0,0.631147,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.292225,0.0,0.0,0.529955,0.0,0.0,0.645119,0.000000,0.0,0.466448,0.000000,0.0
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50091,0.0,0.0,0.000000,0.000000,0.440955,0.671256,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.595797,0.0,0.000000,0.000000,0.0
50092,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.378841,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.925462,0.0
50093,0.0,0.0,0.000000,0.000000,0.486407,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.873732,0.000000,0.0,0.000000,0.000000,0.0
50094,0.0,0.0,0.707986,0.706226,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0


Now let's calculate the dot product of the tfidf_martix with itself in order to get a cosine similarity matrix.

In [43]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

 What do we expect the dimensions of this matrix to be?

In [44]:
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.37728826, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.        , 0.37728826, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        1.        ]])

Notice that this matrix also has 1's along the diagonals. Why is that?

In [45]:
cosine_sim.shape

(50096, 50096)

The next part you can Test Drive because now we'll add logic to grab the top 30 movie titles by index

We can build this in a separate module and import it here to see some results

In [46]:
from analysis.utils.recommendation import get_similar_movies

similar_movies = get_similar_movies('Toy Story (1995)', cosine_sim, movies_with_ratings_combined_df)
similar_movies

NameError: name 'movies_with_ratings_combined_df' is not defined

THAT'S IT! :)